In [88]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import urllib#pour récupérer les données
import bs4#pour rendre lisibles les données
import re
import time

Récupération des députés sur le site de l'Assemblée Nationale

In [89]:
def Scrap (url):
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    page = bs4.BeautifulSoup(html, "lxml")
    return page

In [90]:
def ToDf (table) :
    Listgrossiere=pd.read_html(str(table))#on transforme le tableau en liste de dataframes
    listeDf=Listgrossiere[0]#on récupère le dataframe
    return listeDf

In [104]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str(0)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
table = page.find('table')
liste=ToDf(table)
listelien=[]
for i in range(32):
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str((i+1)*500)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
    table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
    listeDf=ToDf(table)
    liste=pd.concat([liste,listeDf])#on récupère le dataframe et on l'ajoute à la liste  
    for ele in table.findAll('a'):
        listelien.append(ele.get('href'))
liste=liste.reset_index()#on recrée un index afin de ne pas s'arrêter à 500 puis recommencer comme dans les pages web
listeDepute=liste[['Nom','Né le']]#on récupère les variables qui nous intéressent  

In [105]:
len(listelien)

15860

In [92]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/0000/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
listeDf=ToDf(table)
"""j'ai mis ce bout de code dans la fonction au-dessus pour qu'il prenne plus que les 500 premiers !"""
#Ce code permet de récupérer les 'href'
listelien=[]
for ele in table.findAll('a'):
    listelien.append(ele.get('href'))
listelien[1]


'/sycomore/fiche/(num_dept)/19112'

In [15]:
listeDepute.to_csv(r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\liste_depute.csv')#on met les données sur csv pour ne pas avoir à rescrapper à chaque fois

On s'intéresse ensuite à la récupération des mandats de chaque député (partie en cours).

In [107]:

def recup_dep_naiss (page):
    depute_naiss=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore-infos-generales"})
    dep_naiss=re.findall('\((\w*-?\w*)\s', str(depute_naiss))#on récupère les départements
    if dep_naiss==['']:
        dep_naiss=['Inconnu']
    df_naissance=pd.DataFrame([['Inconnu', 'Inconnu']], columns=('Département de naissance', 'Département de mort'))
    if dep_naiss :
        if len(dep_naiss)==2 :#on connait plus les départements de naissance que les morts (pour les députés vivants au moins)
            df_naissance=pd.DataFrame([dep_naiss], columns=('Département de naissance', 'Département de mort'))
        elif len(dep_naiss)==1 :
            dep_complete=[dep_naiss[0],'Inconnu']
            df_naissance=pd.DataFrame([dep_complete], columns=('Département de naissance', 'Département de mort'))
       
    return(df_naissance)

In [94]:
def scrap_un_mandat(mandat):
    #On récupère et lave les titres des colonnes
    columns=mandat.findAll('dt')
    columns=re.sub('<[^<]+?>', '', str(columns))  
    
    #On récupère et lave les titres des lignes
    lines=mandat.findAll('dd')
    lines=re.sub('<[^<]+?>', '', str(lines))
    lines=re.sub('\t', '', str(lines))
    lines=re.sub("\n", '', str(lines))
    line=''
    
    
    #On met tout dans un dataframe
    columns=columns[1:-1].split(',')
    lines=lines[1:-1].split(',')
    df=pd.DataFrame([lines],columns=columns)
    return df

In [95]:
def RecupNom (page):
    nom_depute=page.find('title')#on va là où il y a le nom
    nom_depute=re.sub('<[^<]+?>', '', str(nom_depute))#on enlève les balises
    nom_depute=nom_depute.split()#on récupère les mots du titre
    mot=nom_depute[0]#on initialise au premier mot
    nom_dep=''#on initialise la variable où on va récupérer le nom
    while mot!='-' :#on prend les premiers mots qui composent le nom, avant la suite du titre
        nom_dep=nom_dep+' '+mot#on concatène le mot appartenant au nom avec les précédents
        nom_depute=nom_depute[1:]#on supprime le mot étudié
        mot=nom_depute[0]#pour passer au suivant
    nom_dep=nom_dep[1:]#il y a un espace au début, on l'enlève
    return nom_dep#on retourne le mot complet. Il est surement possible de le faire beaucoup plus vite en regex, mais les miennes ne marchent pas.

Cette proposition permet de se passer de RecupNom mais il y a encore des bugs : 
Attention !!!!!!!!!!!!!!!!!!!!!!! La listeDepute n'est pas dans le même ordre que la liste en ligne, donc cela n'attribue pas le nom au bon député ! (d'où le retour à RécupNom)

In [110]:
tps1=time.time()
liste=[]
numProbleme=[]
df_naiss=[]
for i in range(4000, 6000) : #à terme len(listelien)
    if i%100==1 :#pour mesurer l'avancement du scrapping
        print(i)
    page=Scrap('http://www2.assemblee-nationale.fr/'+listelien[i])
    #Une petite blague c'est glissé dans le site : les mandats de sénateurs sont présent dans le code html mais ils sont "cachés"
    #On les enlève donc d'abord
    pouf=page.find('div', {'id':"mandats_an"})
    df_naissance=recup_dep_naiss(page)#on récupère d'abord les départements de naissance et de mort du député    
    df_naissance['Nom']=RecupNom(page)
    if len(df_naiss)==0:
        df_naiss=df_naissance
    else :
        df_naiss=pd.concat([df_naiss, df_naissance])
    if pouf :
        depute=pouf.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    else :#éviter les cas d'assemblée nationale constituante qui ont un autre nom que mandats_an
        depute=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    for mandat in depute :   
        try:
            df = scrap_un_mandat(mandat)
        except AssertionError :
            numProbleme.append(i)
        else :#ne marche pas en elif ! A cause du try
            df['Nom']=RecupNom(page)            
            if len(liste)==0: #On initialise avec le premier mandat
                liste=df                  
            else:
                liste=pd.concat([liste,df])#on fusionne les mandats
                
tps2=time.time()  
tps=tps2-tps1    
print(len(numProbleme))
print("temps d'exécution",tps)   
display(liste.head(5))
df_naiss.head()

4001


C:\Users\sylva\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6
temps d'exécution 591.0180802345276


,Département,Groupe,Législature,Mandat,Nom,Régime politique
0,Saône-et-Loire,Indépendants et paysans d'action sociale,Ire législature,Du 30 novembre 1958 au 9 octobre 1962,Pierre Dufour,Cinquième République - Assemblée nationale
0,Finistère,Union pour la nouvelle République-UDT,IIe législature,Du 25 novembre 1962 au 2 avril 1967,Gabriel Miossec,Cinquième République - Assemblée nationale
0,Finistère,Union démocratique pour la V° République,IIIe législature,Du 12 mars 1967 au 30 mai 1968,Gabriel Miossec,Cinquième République - Assemblée nationale
0,Finistère,Union des démocrates pour la République,IVe législature,Du 30 juin 1968 au 1er avril 1973,Gabriel Miossec,Cinquième République - Assemblée nationale
0,Lot-et-Garonne,Républicains de gauche,XIVe législature,Du 29 avril 1928 au 31 mai 1932,"Pierre, Albert, Jean, Martial Tricard-Graveron",Troisième République - Chambre des députés


,Département de naissance,Département de mort,Nom
0,Inconnu,Inconnu,Pierre Dufour
0,Finistère,Lieu,Gabriel Miossec
0,Paris,Inconnu,"Pierre, Albert, Jean, Martial Tricard-Graveron"
0,Haute-Savoie,Essonne,Armand Cachat
0,Vendée,Loire-Atlantique,Jean Martineau


Il y des pages de députés jusqu'à 19500 sur le site.

In [ ]:
#On joint les deux dataframes ayant des informations personnelles sur les députés
listeDepu=listeDepute.copy()
df_naiss2=df_naiss.copy()
listeDepute_complete=listeDepu.merge(df_naiss2, on='Nom', how='outer')
listeDepute_complete

In [ ]:
liste=liste.rename(columns={' Département': 'Département', ' Groupe': 'Groupe', ' Législature': 'Législature', ' Mandat' :'Mandat'})

In [ ]:
liste_dep=liste[['Département', 'Groupe', 'Législature', 'Mandat', 'Régime politique', 'Nom']]

In [ ]:
len(liste)/len(numProbleme)#un indicateur du nombre de problèmes, pour vérifier que les résultats sont encore significatifs

*Formatage des dates

In [ ]:
debut_mandat=[]
for dates in df["Mandat"]:
    dates=str(dates).split()
    debut_mandat.append(' '.join(dates[1:4]))
    
df["Debut Mandat"]=np.array(debut_mandat)
df.head()

In [ ]:
liste_dep.to_csv(r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\liste_mandat.csv')
listeDepute_complete.to_csv(r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\listeDepute_complete.csv')
#on met les données sur csv pour ne pas avoir à rescrapper à chaque fois